In [ ]:
import os
from pathlib import Path


import pandas as pd
import matplotlib.pyplot as plt
from astropy.table import Table, join
from scipy.stats import binned_statistic
import numpy as np

from utils import better_step

In [ ]:
params = {
    "legend.fontsize": "x-large",
    "axes.labelsize": "x-large",
    "axes.titlesize": "x-large",
    "xtick.labelsize": "x-large",
    "ytick.labelsize": "x-large",
    "figure.facecolor": "w",
    "xtick.top": True,
    "ytick.right": True,
    "xtick.direction": "in",
    "ytick.direction": "in",
    "font.family": "serif",
    "mathtext.fontset": "dejavuserif",
}
plt.rcParams.update(params)

In [ ]:
vi_results = pd.read_csv(Path(os.environ["SCRATCH"]) / "DESI-II-VI-results" / "merged_vi.csv" )

In [ ]:
cat = Table.read(Path(os.environ["CFS"]) / "desi/users/bid13/DESI_II/pilot_obs/merged_cat.fits")

In [ ]:
cat = join(Table.from_pandas(vi_results),cat,keys="TARGETID",join_type="left")
cat["success"] = cat["VI_quality"]>2

In [ ]:
def make_success_plot(cat,colname="mag_i", nbins=10, ax =None):
    success, bin_edges , _ = binned_statistic(cat[colname], cat["success"],bins=nbins,statistic="mean")
    count, bin_edges, _ = binned_statistic(cat[colname], cat["success"],bins=nbins,statistic="count")
    e_success = np.sqrt(success*(1-success)/count)
    ax = better_step(bin_edges, success, yerr=e_success, ax = ax)
    ax.set_ylim(0.45,1)
    # ax.grid(linestyle="--")
    return ax

In [ ]:
###Figure out how to merge VI spectypes
cat = cat[cat["Z"]>0.01]

In [ ]:
ax = make_success_plot(cat)
ax.set_title("All Objects")
ax.set_xlabel("$i$-mag")

In [ ]:
ax = make_success_plot(cat, colname="mag_i_fiber")
ax.set_title("All Objects")
ax.set_xlabel("$i$-fiber-mag")

In [ ]:
mag_bin_edges = [22,22.5,23,23.75, 24.5]
fig, axs = plt.subplots(2,2, figsize=(14,12))
axs = np.ravel(axs)

for i in range(4):
    cat_sel = cat[(cat["mag_i"]>=mag_bin_edges[i]) & (cat["mag_i"]<mag_bin_edges[i+1])]
    make_success_plot(cat_sel, ax= axs[i])
    axs[i].set_title(f"{mag_bin_edges[i]}$\leq i <${mag_bin_edges[i+1]}")
    axs[i].axhline(np.mean(cat_sel["success"]), ls="--", c="k", alpha=0.5)
    axs[i].set_xlabel("$i$-mag")

In [ ]:
# mag_bin_edges = [22,22.75,23.25,24, 24.75]
mag_bin_edges = [22,22.5,23,23.75, 24.5]
fig, axs = plt.subplots(2,2, figsize=(14,12))
axs = np.ravel(axs)

for i in range(4):
    cat_sel = cat[(cat["mag_i_fiber"]>=mag_bin_edges[i]) & (cat["mag_i_fiber"]<mag_bin_edges[i+1])]
    make_success_plot(cat_sel,colname="mag_i_fiber", ax= axs[i])
    axs[i].set_title(f"{mag_bin_edges[i]}$\leq i <${mag_bin_edges[i+1]}")
    axs[i].axhline(np.mean(cat_sel["success"]), ls="--", c="k", alpha=0.5)
    axs[i].set_xlabel("$i$-mag")

In [ ]:
mask = cat["success"] & (cat["mag_i"]<23)
plt.hexbin(cat["mag_i"][mask], np.log10(cat["EXPTIME"][mask]), cmap="Blues")